# Dipendenze

In [18]:
%pip install --quiet --upgrade python-dotenv pypdf langchain-openai langchain langchain-core langchain-text-splitters langchain-community langgraph

Note: you may need to restart the kernel to use updated packages.


### LangSmith Setup

In [ ]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"

In [31]:
#Env File

from dotenv import load_dotenv

load_dotenv("intro.env");

### OAI LLM Setup

In [29]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)

### Embeddings setup

In [30]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=os.getenv("AZURE_OPENAI_EMBEDDINGS_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT_NAME"),
    openai_api_version=os.getenv("AZURE_OPENAI_EMBEDDINGS_API_VERSION"),
)

### VectorStorage

In [16]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

# Progetto

### 1 - Retrieval

In [8]:
DIR_PATH = "./data"

In [9]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
import pypdf

loader = PyPDFDirectoryLoader(DIR_PATH)
docs = loader.load()

In [10]:
print(len(docs[0].page_content))

1375


In [ ]:
#Chunking

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # chunk size (characters)
    chunk_overlap=100,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)

all_splits = text_splitter.split_documents(docs)

print(f"Split into {len(all_splits)} sub-documents.")

Split into 36 sub-documents.


In [17]:
ids = vector_store.add_documents(documents=all_splits)

for i in range(3):
    print(ids[i])

657ee2e3-eaa9-433c-8931-6dd61927d86b
1e3c0db8-2b9e-4d72-b7b5-5abd771cf0bc
052beaa8-cd0b-4efc-a1b6-4e7ac5a03ddc


### 2 - Retrieval & Generation